In [1]:
excel_path = '/dls/e02/data/2020/cm26481-1/processing/Merlin/20200130_80kV_graphene_600C_pty/20200130_Ptychography_speadsheet.xlsx'
session_path = '/dls/e02/data/2020/cm26481-1/processing/Merlin/20200130_80kV_graphene_600C_pty/'
header = 7
cols = "A:R" 

In [2]:
import pandas as pd
import os
import h5py
import pyxem as pxm
import numpy as np
import sys
sys.path.append('/dls/science/groups/e02/Mohsen/code/Git_Repos/Merlin-Medipix/')
import epsic_tools.api as epsic

In [61]:
def purge_file(parent_path, name_has, extension):
    """
    to conveniently delete unwated files in a nested folder.
    Input
    _______
    path: str
        full path of the parent folder
    name_has: str
        string appearing in the file name
    extension: str
        unwated files extension
    
    """
    to_del = []
    
    for root, dirs, files in os.walk(parent_path):
        path = root.split(os.sep)
        for file in files:
            if (file.endswith(extension) and name_has in file):
                to_del.append(os.path.join(os.sep,*path,file))
    
    for file in to_del:
        if os.path.isfile(file):
            os.remove(file)
            print('deleted: ', file)
    return 
    

In [63]:
def parse_excel(excel_path):
    data = pd.read_excel(excel_path, header=header, usecols=cols)
    # remove rows where all elements are nans
    data.dropna(how='all', inplace=True) 
    
    list_of_dicts = []
    for i in range(len(data)):
        entry = data.iloc[i]
        entry_dict = entry.to_dict()
        list_of_dicts.append(entry_dict)
    return list_of_dicts

In [64]:
def _create_metadata_dict(session_path, entry_dict):
    metadata_dict = {}
    
    metadata_dict['dataset'] = entry_dict['Folder']
    metadata_dict['accelerating voltage (keV)'] = entry_dict.pop('keV')
    metadata_dict['A2 value (keV)'] = entry_dict.pop('A2 (kV)')
    metadata_dict['spot size'] = entry_dict.pop('Spot')
    metadata_dict['CL aperture'] = entry_dict.pop('CLA')
    metadata_dict['convergence simi-angle'] = {}
    metadata_dict['convergence simi-angle']['nominal (mrad)'] = entry_dict.pop('alpha')
    metadata_dict['convergence simi-angle']['calibrated (mrad)'] = []
    metadata_dict['convergence simi-angle']['calibration comments'] = []
    metadata_dict['camera length'] = {}
    metadata_dict['camera length']['nominal (cm)'] = entry_dict.pop('CL')
    metadata_dict['camera length']['calibrated (cm)'] = []
    metadata_dict['camera length']['calibration comments'] = []
    metadata_dict['defocus'] = {}
    metadata_dict['defocus']['nominal (nm)'] = entry_dict.pop('nominal defocus (nm)')
    metadata_dict['defocus']['calibrated (nm)'] = []
    metadata_dict['defocus']['calibration comments'] = []
    metadata_dict['MAG'] = entry_dict.pop('Scan Mag')
    metadata_dict['FOV (A)'] = entry_dict.pop('Field of view (Ang)')
    metadata_dict['recorded probe positions'] = entry_dict.pop('probe positions recorded')
    metadata_dict['counter depth'] = entry_dict.pop('counter depth')
    metadata_dict['saved data bit depth'] = entry_dict.pop('Saved data bit depth')
    metadata_dict['step size (A)'] = entry_dict.pop('step size (Ang)')
    metadata_dict['frame time (ms)'] = entry_dict.pop('frame time (ms)')
    
    metadata_dict['Comments'] = entry_dict.pop('Notes')
    
    return metadata_dict

In [ ]:
list_of_dicts = parse_excel(excel_path)

for entry in list_of_dicts:
    results = []
    if entry['Folder'] in os.listdir(session_path):
        metadata_dict = _create_metadata_dict(session_path, entry)
        results.append(metadata_dict)
        hdf5_files = _get_hdf_list(os.path.join(session_path, entry['Folder']))
        for file in hdf5_files:
            a, b = os.path.splitext(file)
            meta_file_name = a + '_metadata.hdf5'
            epsic.ptycho_utils.save_dict_to_hdf5(metadata_dict, meta_file_name)
    else:
        print(entry['Folder'] , ' not among the datasets - no metadata added.')

In [65]:
test = epsic.ptycho_utils.load_dict_from_hdf5('/dls/e02/data/2020/cm26481-1/processing/Merlin/20200130_80kV_graphene_600C_pty/20200130 163208/binned_diff_80kV_600C_CLA_40um_CL_10cm_8C_20Mx_A2_4p71_dfneg10nm_scan_array_255by255_diff_plane_128by128__metadata.hdf5')

In [66]:
test

{'A2 value (keV)': 4.71,
 'CL aperture': '40um',
 'Comments': nan,
 'FOV (A)': 9.52e-09,
 'MAG': '20Mx',
 'accelerating voltage (keV)': 80.0,
 'camera length': {'calibrated (cm)': array([], dtype=float64),
  'calibration comments': array([], dtype=float64),
  'nominal (cm)': 17.32},
 'convergence simi-angle': {'calibrated (mrad)': array([], dtype=float64),
  'calibration comments': array([], dtype=float64),
  'nominal (mrad)': 31.74},
 'counter depth': 6.0,
 'dataset': '20200130 163208',
 'defocus': {'calibrated (nm)': array([], dtype=float64),
  'calibration comments': array([], dtype=float64),
  'nominal (nm)': -10.0},
 'frame time (ms)': 0.7,
 'recorded probe positions': 256.0,
 'saved data bit depth': 8.0,
 'spot size': '8C',
 'step size (A)': 3.71875e-11}